# Native method
In this case, we assume the stock timeseries to be continuous
(the price tomorrow are likely to be close to the price today) as
well as periodical with a daily period. Thus a common-sense approach is to always
predict that the Closing price tomorrow will be equal to the price
today.

In order to get the "Target" to calculate "Rank", two days prices need to be predicted. So we will predict prices of tomorrow and the day after tomorrow. We assume prices of next two days are equal to last two days.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
stock_prices = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv', index_col=False)
print(f"Original stock prices: {len(stock_prices)}")
cleaned_stock_prices = stock_prices[stock_prices['Close'].notna()]
print(f"Cleaned stock prices have: {len(cleaned_stock_prices)}")

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files

In [ ]:
tag = 0
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    if tag == 0:
        train_series_for_predict = cleaned_stock_prices[cleaned_stock_prices['Date'] > '2021-01-01']
        target_series = train_series_for_predict
        tag = 1
        
    cleaned_prices = prices[prices['Close'].notna()]
    target_series = target_series.append(cleaned_prices)
    new_securities = target_series.groupby('SecuritiesCode')['Close'].apply(list)
    security_code = []
    next_day = []
    next2_day = []
    
    for idx, value in new_securities.items():
        s_id = idx
        s_v = value
        security_code.append(s_id)
    
        next_day.append(s_v[-2])
        next2_day.append(s_v[-1])
        
    tuple_list = list(zip(security_code, next_day, next2_day))

    next_day_df = pd.DataFrame(tuple_list, columns=['SecuritiesCode', 'NextDay', "Next2Day"])
    new_df = pd.merge(prices, next_day_df, on="SecuritiesCode")
    new_df['Target'] = (new_df['Next2Day'] - new_df['NextDay']) / new_df['NextDay']
    new_df['Rank'] = new_df['Target'].rank(ascending=False, method='first') - 1
    new_df = new_df.sort_values('Rank').reset_index(drop=True)
    new_df['Rank'] = new_df['Rank'].astype('int')
    rankdict = dict(zip(new_df["SecuritiesCode"],new_df["Rank"]))
    sample_prediction['Rank'] = sample_prediction["SecuritiesCode"].map(rankdict)
    env.predict(sample_prediction)